<a href="https://colab.research.google.com/github/ToyTeX/ToyTeX/blob/main/HC_ShapeInterpolationJulia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Homotopy Continuation](https://www.juliahomotopycontinuation.org/) "Shape Interpolation"  Notebook


In [2]:
import Pkg
Pkg.add("HomotopyContinuation")
Pkg.add("DynamicPolynomials")
Pkg.add("GLMakie")
Pkg.add("Plots")
Pkg.add("LinearAlgebra")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
Precompiling project...
   4218.0 ms  ✓ LuxLib → LuxLibLoopVectorizationExt
  1 dependency successfully precompiled in 10 seconds. 558 already precompiled.
   Resolving package versions...
    Updating `~/.julia/environments/v1.11/Project.toml`
  [7c1d4256] + DynamicPolynomials v0.6.2
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
   Installed GLFW ───── v3.4.4
   Installed ModernGL ─ v1.1.8
   Installed GLMakie ── v0.13.4
   Installed MeshIO ─── v0.5.3
    Updating `~/.julia/environments/v1.11/Project.toml`
  [e9467ef8] + GLMakie v0.13.4
    Updating `~/.julia/environments/v1.11/Manifest.toml`
  [f7f18e0c] + GLFW v3.4.4
  [e9467ef8] + GLMakie v0.13.4
  [7269a6da] + MeshIO v0.5.3
  [66fc600b] + ModernGL v1.1.8
Precompiling project...
   7195.4 ms  ✓ ModernGL
   5526.2 ms  ✓ GLFW
   7888.7 ms

# [Minimal Example](https://www.juliahomotopycontinuation.org/guides/introduction/): Solving a homotopy system


In [1]:
using HomotopyContinuation
using DynamicPolynomials
# declare variables x and y
@var x y
# define the polynomials
f₁ = (x^4 + y^4 - 1) * (x^2 + y^2 - 2) + x^5 * y
f₂ = x^2+2x*y^2 - 2y^2 - 1/2
F = System([f₁, f₂])
result = solve(F)


[ Info: Precompiling HomotopyContinuation [f213a82b-91d6-5c5d-acf7-10f1c761b327] 


LoadError: ArgumentError: Package DynamicPolynomials not found in current path.
- Run `import Pkg; Pkg.add("DynamicPolynomials")` to install the DynamicPolynomials package.

In [ ]:
real_solutions(result)

4-element Vector{Vector{Float64}}:
 [-0.93689796679633, 0.31228408173860106]
 [0.8209788924342627, -0.697132645948946]
 [-1.671421392838003, 0.6552051858720408]
 [0.8999179208471727, -1.2441827613422727]

# Trivial Example:  0 solutions for 2 circles in 4D


1.    A circle (x² + y² = 1) with parametric extensions u = x - x³, v = y - y³  
2.   Same circle (x² + y² = 1) with different parametric extensions u = 2x² - 1,
   v = 3y - 4y³






In [1]:
@var x y u v
###Circle in 4 variables
f_1 = [x^2 + y^2 - 1, u - x + x^3, v - y + y^3]
###Circle in 4 variables
g_1 = [x^2 + y^2 - 1, u - 2x^2 + 1, v - 3y + 4y^3]
###Define and solve system.
H_1 = System([f_1..., g_1...])
result = solve(H_1)

LoadError: LoadError: UndefVarError: `@var` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
in expression starting at In[1]:1





*   Even though both curves project to the unit circle when you look at just the (x,y) coordinates, their **(u,v) components follow completely different rules**
*  While the curves intersect in the (x,y) projection, they **never occupy the same point in the full 4D (x,y,u,v) space**





# Ellipse to Circle
Defines a symbolic polynomial system H(x, y, t) that linearly interpolates between the circle and the ellipse.

At `t = 0, H(x, y, 0) = circle_eq`

At `t = 1, H(x, y, 1) = ellipse_eq`

This is a valid homotopy in HomotopyContinuation.jl. We can now use it to:

Solve for specific values of t

Track solutions as t moves from 0 to 1

In [ ]:
using HomotopyContinuation
using DynamicPolynomials

# Declare symbolic variables
@polyvar x y t

# Define parameters for the ellipse
a = 2.0
b = 1.0

# Define start polynomial system (unit circle)
circle_eq = x^2 + y^2 - 1

# Define target polynomial system (ellipse)
ellipse_eq = (x^2)/(a^2) + (y^2)/(b^2) - 1

F_1 =  System([ellipse_eq, circle_eq])

result = solve(F_1)


Result with 2 solutions
• 4 paths tracked
• 0 non-singular solutions (0 real)
• 2 singular solutions (2 real)
• random_seed: 0x9a9849d0
• start_system: :polyhedral
• multiplicity table of singular solutions:
╭───────┬───────┬────────┬────────────╮
│ mult. │ total │ # real │ # non-real │
├───────┼───────┼────────┼────────────┤
│   2   │   2   │   2    │     0      │
╰───────┴───────┴────────┴────────────╯


The 2 real singular solutions occur because  of **intersection**:  The circle (radius 1) and the ellipse (semi-major axis a=2, semi-minor axis b=1) **intersect at exactly 2 points**. At these locations, the gradients of both curves are parallel (or one is zero).  At the intersection points, both curves have the same tangent line, meaning the system loses rank. The solver detects this as a singular solution  

Since the ellipse has the same minor axis as the circle's radius (b=1) but a larger major axis (a=2), they intersect where the circle crosses the ellipse along the y-axis.

# By Explicit Start Solution

In [ ]:
using HomotopyContinuation
using DynamicPolynomials
using Plots

# Declare symbolic variables
@polyvar x y t

# Define parameters for the ellipse
a = 2.0
b = 1.0

# Define start polynomial system (circle)
circle_eq = x^2 + y^2 - 1

# Define target polynomial system (ellipse)
ellipse_eq = (x^2)/(a^2) + (y^2)/(b^2) - 1

F_2 =  System([ellipse_eq, circle_eq]; variables=[x,y])





System of length 2
 2 variables: x, y

 -1.0 + 0.25*x^2 + 1.0*y^2
 -1.0 + 1.0*x^2 + 1.0*y^2

In [ ]:
# Generate start solutions for the circle equation (parametrized unit circle)
θ_vals = range(0, 2π, length=50)
# Generate start solutions as a vector of standard Julia vectors
start_solutions = [[cos(θ), sin(θ)] for θ in θ_vals]

50-element Vector{Vector{Float64}}:
 [1.0, 0.0]
 [0.9917900138232462, 0.127877161684506]
 [0.9672948630390295, 0.25365458390950735]
 [0.9269167573460217, 0.3752670048793741]
 [0.8713187041233894, 0.49071755200393785]
 [0.8014136218679566, 0.598110530491216]
 [0.7183493500977276, 0.6956825506034864]
 [0.6234898018587336, 0.7818314824680298]
 [0.5183925683105252, 0.8551427630053461]
 [0.4047833431223938, 0.9144126230158125]
 [0.28452758663103245, 0.9586678530366606]
 [0.15959989503337932, 0.9871817834144501]
 [0.03205157757165533, 0.9994862162006879]
 ⋮
 [0.1595998950333793, -0.9871817834144502]
 [0.284527586631032, -0.9586678530366608]
 [0.4047833431223937, -0.9144126230158125]
 [0.5183925683105245, -0.8551427630053464]
 [0.6234898018587334, -0.7818314824680299]
 [0.7183493500977277, -0.6956825506034863]
 [0.8014136218679564, -0.5981105304912162]
 [0.8713187041233894, -0.49071755200393785]
 [0.9269167573460216, -0.3752670048793746]
 [0.9672948630390293, -0.2536545839095075]
 [0.99179001

In [ ]:
solve(F_2, start_solutions)

Result with 2 solutions
• 4 paths tracked
• 0 non-singular solutions (0 real)
• 2 singular solutions (2 real)
• random_seed: 0xfea3bcd1
• start_system: :polyhedral
• multiplicity table of singular solutions:
╭───────┬───────┬────────┬────────────╮
│ mult. │ total │ # real │ # non-real │
├───────┼───────┼────────┼────────────┤
│   2   │   2   │   2    │     0      │
╰───────┴───────┴────────┴────────────╯


# Circle to Torus Homotopy Continuation
 Constructing a homotopy system between a torus and a circle requires some care because these shapes generally live in different dimensions.

1.  A torus is a 2D surface embedded in 3D.  
2.  A circle is a 1D curve in 2D.   

So to construct a meaningful homotopy continuation between them we need to make the spaces compatible.  Let's view the torus and circle as surfaces in 3D, and deform one into the other via a homotopy in 3D space.  We use random affine constraint to ensure the system is solvable.   


*   t = 0; circle in xy-plane, z = 0.
*   t = 1; torus in 3D



In [ ]:
using HomotopyContinuation
using DynamicPolynomials

@polyvar x y z t

# Parametize Torus
R = 2.0
r = 1.0

# Torus
F_torus_poly = ((x^2 + y^2 + z^2 + R^2 - r^2)^2 - 4R^2 * (x^2 + y^2))

# Circle embedded in 3D (in xy-plane at z = 0)
F_circ = x^2 + y^2 - 1
plane_eq = z



# Define Homotopy
H_1 = (1 - t) * F_circ + t * F_torus_poly
H_2 = (1 - t) * plane_eq
a, b, c, d = randn(4)
H_3 = a*x + b*y + c*z + d
l, m, n, k = randn(4)
H_4 = l*x + m*y + n*z + k

F_3 = System([H_1, H_2, H_3, H_4])



System of length 4
 4 variables: x, y, z, t

 -1.0 + 10.0*t - 11.0*t*x^2 + 1.0*t*x^4 - 11.0*t*y^2 + 1.0*t*y^4 + 6.0*t*z^2 + 1.0*t*z^4 + 2.0*t*x^2*y^2 + 2.0*t*x^2*z^2 + 2.0*t*y^2*z^2 + 1.0*x^2 + 1.0*y^2
 1.0*z - 1.0*t*z
 0.669919728195185 - 1.1495865693437*x + 0.472906029671582*y - 0.404408342121351*z
 0.72806350251845 + 0.167103381333705*x + 1.10180488816297*y - 0.127682297763651*z

In [ ]:
θ_vals = range(0, 2π, length=30)
start_solutions2 = [[cos(θ), sin(θ), 0.0] for θ in θ_vals]
println("Start solutions found: ", length(start_solutions))
solve(F_3, start_solutions2)

Start solutions found: 30


Result with 5 solutions
• 5 paths tracked
• 5 non-singular solutions (1 real)
• random_seed: 0xaf6fe582
• start_system: :polyhedral


Each point on the circle  moves to the torus surface.  However, the circle is a 1D manifold and the torus is a 2D manifold, so this homotopy only maps a 1D subset of the torus.   

While a continuous map can be constructed that maps the circle into the torus, the circle cannot be continuously deformed to cover or be topologically equivalent to the torus.


The homotopy construction [illustrates a continuous embedding of the circle that gradually takes the shape of a loop on the torus](https://claude.ai/public/artifacts/badbccd3-5ead-4f0d-a0e9-5829e18e36c0), or perhaps even a path that traces a portion of the torus's surface, rather than a deformation of the entire circle into the entire torus.

[Visual](https://claude.ai/public/artifacts/e91eb648-2082-4021-a302-17bfa3a475ab) Implementation Details:

* Marching Cubes Approach: Instead of parametric surfaces, it finds the zero set of the homotopy system using a threshold-based method
* Gradient Normals: Computes surface normals using numerical gradients of the implicit function
* Real-time Computation: The surface is recomputed for each t value, showing the actual algebraic deformation

[Visual 2](https://claude.ai/public/artifacts/beb44113-3ad6-48cb-ba44-b5d4cb4e1a86)

# Point Clouds

In [3]:
using Random
using GLMakie

function generate_torus_point_cloud(R::Float64, r::Float64, N::Int)

[ Info: Precompiling GLMakie [e9467ef8-e4e7-5192-8a1a-b1aee30e663a] 
┌ Warning:     OpenGL/GLFW wasn't loaded correctly or couldn't be initialized.
│     This likely means, you're on a headless server without having OpenGL support setup correctly.
│     Have a look at the troubleshooting section in the readme:
│     https://github.com/MakieOrg/Makie.jl/tree/master/GLMakie#troubleshooting-opengl.
└ @ GLMakie ~/.julia/packages/GLMakie/WBusz/src/gl_backend.jl:4
ERROR: LoadError: InitError: Exception[GLFW.GLFWError(65550, "X11: The DISPLAY environment variable is missing"), ErrorException("glfwInit failed")]
Stacktrace:
  [1] __init__()
    @ GLFW ~/.julia/packages/GLFW/a10jC/src/GLFW.jl:69
  [2] run_module_init(mod::Module, i::Int64)
    @ Base ./loading.jl:1378
  [3] register_restored_modules(sv::Core.SimpleVector, pkg::Base.PkgId, path::String)
    @ Base ./loading.jl:1366
  [4] _include_from_serialized(pkg::Base.PkgId, path::String, ocachepath::String, depmods::Vector{Any}, ignore_nati

LoadError: Failed to precompile GLMakie [e9467ef8-e4e7-5192-8a1a-b1aee30e663a] to "/root/.julia/compiled/v1.11/GLMakie/jl_tkblNT".